SynchroSqueezing

In [ ]:
!pip install mne
!pip install ssqueezepy
!pip install stockwell
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.0/593.0 kB 23.6 MB/s eta 0:00:00


Importing necessary libraries

In [ ]:
import matplotlib.pyplot as plt
import os
os.environ['SSQ_GPU'] = '1'
from ssqueezepy import ssq_cwt
import numpy as np
from stockwell import st
from sklearn.preprocessing import MinMaxScaler
import mne
from scipy.ndimage import zoom
import tensorflow as tf

Save TFR images

In [ ]:
def save_tfr_image(tfr_matrix, class_name, image_name, save_dir, img_size=(224, 224)):
    # Ensure the class directory exists
    class_dir = os.path.join(save_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    # Calculate the zoom factors for resizing
    zoom_factors = (img_size[0] / tfr_matrix.shape[0], img_size[1] / tfr_matrix.shape[1])

    # Resize the TFR matrix using scipy's zoom
    resized_img = zoom(tfr_matrix, zoom_factors)

    # Normalize the matrix for better visualization (0-255 range for grayscale)
    normalized_img = (resized_img - resized_img.min()) / (resized_img.max() - resized_img.min()) * 255
    normalized_img = normalized_img.astype(np.uint8)

    # Save the image using matplotlib
    image_path = os.path.join(class_dir, f"{image_name}.png")
    plt.imsave(image_path, normalized_img,cmap = 'viridis')
    print(f"Saved {image_path}")

Dataset

In [ ]:
# Poland Dataset
!wget https://repod.icm.edu.pl/api/datasets/251/versions/59/files/download?format=original -O dataverse_files.zip

# Extract the dataset
!unzip dataverse_files.zip -d dataverse_files

In [ ]:
directory='dataverse_files/'
output_directory = "./data_15sec/"

Generate TFR images

In [ ]:
edf_files = [f for f in os.listdir(directory) if f.endswith('.edf')]
# edf_files = ['s14.edf']
window_size = 15

def generate_tfrs():
    for edf_file in edf_files:
            print(edf_file)
            file_path = os.path.join(directory, edf_file)
            raw = mne.io.read_raw_edf(file_path, preload=True)

            # Preprocessing
            raw.notch_filter([50, 60], fir_design='firwin')
            raw.filter(l_freq=0.5, h_freq=45)

            # Standardization
            scaler = mne.decoding.Scaler(info=raw.info)
            data = scaler.fit_transform(raw.get_data().T).T[0]

            print(data.shape)
            # # Segment into windows
            sfreq = raw.info['sfreq']
            window_length = int(window_size * sfreq)

            for i in range(0, data.shape[1] - window_length + 1, window_length):
                for k in range(19):
                    # i = 0*window_length
                    # k = 14
                    # print(window_length)
                    window_data = data[k, i:i + window_length]
                    Twxo, Wxo, *_ = ssq_cwt(window_data, astensor=True)
                    window_tfr = np.abs(Twxo.cpu().data.numpy())
                    # plt.imshow(window_tfr, aspect='auto', cmap='turbo')
                    # plt.show()
                    save_tfr_image(window_tfr, 'schizophrenic' if edf_file.startswith('s') else 'healthy', edf_file+str(i//window_length)+'_'+str(k), output_directory)

In [ ]:
generate_tfrs()

s08.edf
Extracting EDF parameters from /content/dataverse_files/s08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227749  =      0.000 ...   910.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition band

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


(19, 227750)
Saved ./data_15sec/schizophrenic/s08.edf0_0.png
Saved ./data_15sec/schizophrenic/s08.edf0_1.png
Saved ./data_15sec/schizophrenic/s08.edf0_2.png
Saved ./data_15sec/schizophrenic/s08.edf0_3.png
Saved ./data_15sec/schizophrenic/s08.edf0_4.png
Saved ./data_15sec/schizophrenic/s08.edf0_5.png
Saved ./data_15sec/schizophrenic/s08.edf0_6.png
Saved ./data_15sec/schizophrenic/s08.edf0_7.png
Saved ./data_15sec/schizophrenic/s08.edf0_8.png
Saved ./data_15sec/schizophrenic/s08.edf0_9.png
Saved ./data_15sec/schizophrenic/s08.edf0_10.png
Saved ./data_15sec/schizophrenic/s08.edf0_11.png
Saved ./data_15sec/schizophrenic/s08.edf0_12.png
Saved ./data_15sec/schizophrenic/s08.edf0_13.png
Saved ./data_15sec/schizophrenic/s08.edf0_14.png
Saved ./data_15sec/schizophrenic/s08.edf0_15.png
Saved ./data_15sec/schizophrenic/s08.edf0_16.png
Saved ./data_15sec/schizophrenic/s08.edf0_17.png
Saved ./data_15sec/schizophrenic/s08.edf0_18.png
Saved ./data_15sec/schizophrenic/s08.edf1_0.png
Saved ./data_15sec

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


(19, 241000)
Saved ./data_15sec/schizophrenic/s03.edf0_0.png
Saved ./data_15sec/schizophrenic/s03.edf0_1.png
Saved ./data_15sec/schizophrenic/s03.edf0_2.png
Saved ./data_15sec/schizophrenic/s03.edf0_3.png
Saved ./data_15sec/schizophrenic/s03.edf0_4.png
Saved ./data_15sec/schizophrenic/s03.edf0_5.png
Saved ./data_15sec/schizophrenic/s03.edf0_6.png
Saved ./data_15sec/schizophrenic/s03.edf0_7.png
Saved ./data_15sec/schizophrenic/s03.edf0_8.png
Saved ./data_15sec/schizophrenic/s03.edf0_9.png
Saved ./data_15sec/schizophrenic/s03.edf0_10.png
Saved ./data_15sec/schizophrenic/s03.edf0_11.png
Saved ./data_15sec/schizophrenic/s03.edf0_12.png
Saved ./data_15sec/schizophrenic/s03.edf0_13.png
Saved ./data_15sec/schizophrenic/s03.edf0_14.png
Saved ./data_15sec/schizophrenic/s03.edf0_15.png
Saved ./data_15sec/schizophrenic/s03.edf0_16.png
Saved ./data_15sec/schizophrenic/s03.edf0_17.png
Saved ./data_15sec/schizophrenic/s03.edf0_18.png
Saved ./data_15sec/schizophrenic/s03.edf1_0.png
Saved ./data_15sec

...

(19, 185000)
Saved ./data_15sec/schizophrenic/s06.edf0_0.png
Saved ./data_15sec/schizophrenic/s06.edf0_1.png
Saved ./data_15sec/schizophrenic/s06.edf0_2.png
Saved ./data_15sec/schizophrenic/s06.edf0_3.png
Saved ./data_15sec/schizophrenic/s06.edf0_4.png
Saved ./data_15sec/schizophrenic/s06.edf0_5.png
Saved ./data_15sec/schizophrenic/s06.edf0_6.png
Saved ./data_15sec/schizophrenic/s06.edf0_7.png
Saved ./data_15sec/schizophrenic/s06.edf0_8.png
Saved ./data_15sec/schizophrenic/s06.edf0_9.png
Saved ./data_15sec/schizophrenic/s06.edf0_10.png
Saved ./data_15sec/schizophrenic/s06.edf0_11.png
Saved ./data_15sec/schizophrenic/s06.edf0_12.png
Saved ./data_15sec/schizophrenic/s06.edf0_13.png
Saved ./data_15sec/schizophrenic/s06.edf0_14.png
Saved ./data_15sec/schizophrenic/s06.edf0_15.png
Saved ./data_15sec/schizophrenic/s06.edf0_16.png
Saved ./data_15sec/schizophrenic/s06.edf0_17.png
Saved ./data_15sec/schizophrenic/s06.edf0_18.png
Saved ./data_15sec/schizophrenic/s06.edf1_0.png
Saved ./data_15sec

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


(19, 241250)
Saved ./data_15sec/healthy/h13.edf0_0.png
Saved ./data_15sec/healthy/h13.edf0_1.png
Saved ./data_15sec/healthy/h13.edf0_2.png
Saved ./data_15sec/healthy/h13.edf0_3.png
Saved ./data_15sec/healthy/h13.edf0_4.png
Saved ./data_15sec/healthy/h13.edf0_5.png
Saved ./data_15sec/healthy/h13.edf0_6.png
Saved ./data_15sec/healthy/h13.edf0_7.png
Saved ./data_15sec/healthy/h13.edf0_8.png
Saved ./data_15sec/healthy/h13.edf0_9.png
Saved ./data_15sec/healthy/h13.edf0_10.png
Saved ./data_15sec/healthy/h13.edf0_11.png
Saved ./data_15sec/healthy/h13.edf0_12.png
Saved ./data_15sec/healthy/h13.edf0_13.png
Saved ./data_15sec/healthy/h13.edf0_14.png
Saved ./data_15sec/healthy/h13.edf0_15.png
Saved ./data_15sec/healthy/h13.edf0_16.png
Saved ./data_15sec/healthy/h13.edf0_17.png
Saved ./data_15sec/healthy/h13.edf0_18.png
Saved ./data_15sec/healthy/h13.edf1_0.png
Saved ./data_15sec/healthy/h13.edf1_1.png
Saved ./data_15sec/healthy/h13.edf1_2.png
Saved ./data_15sec/healthy/h13.edf1_3.png
Saved ./data

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


(19, 236250)
Saved ./data_15sec/healthy/h05.edf0_0.png
Saved ./data_15sec/healthy/h05.edf0_1.png
Saved ./data_15sec/healthy/h05.edf0_2.png
Saved ./data_15sec/healthy/h05.edf0_3.png
Saved ./data_15sec/healthy/h05.edf0_4.png
Saved ./data_15sec/healthy/h05.edf0_5.png
Saved ./data_15sec/healthy/h05.edf0_6.png
Saved ./data_15sec/healthy/h05.edf0_7.png
Saved ./data_15sec/healthy/h05.edf0_8.png
Saved ./data_15sec/healthy/h05.edf0_9.png
Saved ./data_15sec/healthy/h05.edf0_10.png
Saved ./data_15sec/healthy/h05.edf0_11.png
Saved ./data_15sec/healthy/h05.edf0_12.png
Saved ./data_15sec/healthy/h05.edf0_13.png
Saved ./data_15sec/healthy/h05.edf0_14.png
Saved ./data_15sec/healthy/h05.edf0_15.png
Saved ./data_15sec/healthy/h05.edf0_16.png
Saved ./data_15sec/healthy/h05.edf0_17.png
Saved ./data_15sec/healthy/h05.edf0_18.png
Saved ./data_15sec/healthy/h05.edf1_0.png
Saved ./data_15sec/healthy/h05.edf1_1.png
Saved ./data_15sec/healthy/h05.edf1_2.png
Saved ./data_15sec/healthy/h05.edf1_3.png
Saved ./data

Saving generated images to drive

In [ ]:
# Set the locale to UTF-8
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Zip the data folder
!zip -r data_15sec.zip data_15sec

Streaming output truncated to the last 5000 lines.
  adding: data_15sec/healthy/h01.edf27_15.png (deflated 0%)
  adding: data_15sec/healthy/h07.edf59_9.png (deflated 0%)
  adding: data_15sec/healthy/h01.edf1_9.png (deflated 1%)
  adding: data_15sec/healthy/h14.edf34_13.png (deflated 0%)
  adding: data_15sec/healthy/h09.edf23_17.png (deflated 0%)
  adding: data_15sec/healthy/h12.edf15_14.png (deflated 0%)
  adding: data_15sec/healthy/h11.edf26_11.png (deflated 0%)
  adding: data_15sec/healthy/h01.edf60_12.png (deflated 0%)
  adding: data_15sec/healthy/h08.edf4_17.png (deflated 0%)
  adding: data_15sec/healthy/h03.edf9_10.png (deflated 0%)
  adding: data_15sec/healthy/h13.edf25_17.png (deflated 0%)
  adding: data_15sec/healthy/h12.edf29_10.png (deflated 0%)
  adding: data_15sec/healthy/h06.edf2_4.png (deflated 1%)
  adding: data_15sec/healthy/h01.edf1_1.png (deflated 0%)
  adding: data_15sec/healthy/h09.edf14_3.png (deflated 0%)
  adding: data_15sec/healthy/h10.edf5_10.png (deflated 0%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp data_15sec.zip '/content/drive/My Drive/'